In [ ]:
# Import all the necessary libraries
import os
import numpy as np
import torch
import importlib
import skimage as sk
from skimage.color import rgb2hsv
import matplotlib.pyplot as plt
from typing import Callable
import cv2 as cv
from skimage.morphology import closing, opening, disk, remove_small_holes, remove_small_objects

In [220]:
def display_img(img):
    plt.imshow(img, cmap='gray')
    plt.axis('off')
    plt.show()

def display_img_side(img1, img2, idx):
    fig, axs = plt.subplots(1, 2, figsize=(15, 15))
    axs[0].imshow(img1, cmap='gray')
    axs[0].axis('off')
    axs[1].imshow(img2)
    axs[1].axis('off')
    plt.savefig(f'./results/{idx}.png')
    plt.close()

In [ ]:
def remove_objects(img_th, size):
    """
    Remove objects from input image that are smaller than size argument.

    Args
    ----
    img_th: np.ndarray (M, N)
        Image mask of size MxN.
    size: int
        Minimal size of objects

    Return
    ------
    img_obj: np.ndarray (M, N)
        Image after remove small objects operation
    """

    # Define default value for input image
    img_obj = np.zeros_like(img_th)
    # ------------------
    # Your code here ... 
    img_th = img_th.astype(bool)
    img_obj = remove_small_objects(img_th, size)
    # ------------------

    return img_obj

def apply_closing(img_th, disk_size):
    """
    Apply closing to input mask image using disk shape.

    Args
    ----
    img_th: np.ndarray (M, N)
        Image mask of size MxN.
    disk_size: int
        Size of the disk to use for closing

    Return
    ------
    img_closing: np.ndarray (M, N)
        Image after closing operation
    """

    # Define default value for output image
    img_closing = np.zeros_like(img_th)

    # ------------------
    # Your code here ...
    img_closing = closing(img_th, disk(disk_size)) 
    # ------------------

    return img_closing

def apply_closing(img_th, disk_size):
    """
    Apply closing to input mask image using disk shape.

    Args
    ----
    img_th: np.ndarray (M, N)
        Image mask of size MxN.
    disk_size: int
        Size of the disk to use for closing

    Return
    ------
    img_closing: np.ndarray (M, N)
        Image after closing operation
    """

    # Define default value for output image
    img_closing = np.zeros_like(img_th)

    # ------------------
    # Your code here ...
    img_closing = closing(img_th, disk(disk_size)) 
    # ------------------

    return img_closing


def apply_opening(img_th, disk_size):
    """
    Apply opening to input mask image using disk shape.

    Args
    ----
    img_th: np.ndarray (M, N)
        Image mask of size MxN.
    disk_size: int
        Size of the disk to use for opening

    Return
    ------
    img_opening: np.ndarray (M, N)
        Image after opening operation
    """

    # Define default value for output image
    img_opening = np.zeros_like(img_th)

    # ------------------
    # Your code here ... 
    selem = disk(disk_size)
    img_opening = opening(img_th, selem)
    # ------------------

    return img_opening


def apply_median(img, kernel_size):
    """
    Apply median filter to input image.

    Args
    ----
    img: np.ndarray (M, N)
        Image to apply median filter.
    kernel_size: int
        Size of kernel for median filter.

    Return
    ------
    img_median: np.ndarray (M, N)
        Image after applying median filter.
    """

    # Define default value for input image
    img_median = np.zeros_like(img)
    # ------------------
    # Your code here ... 
    img_median = sk.filters.median(img, disk(kernel_size))
    # ------------------

    return img_median

def apply_threshold(img, threshold):
    """
    Apply threshold to input image.

    Args
    ----
    img: np.ndarray (M, N)
        Image to apply threshold.
    threshold: float
        Threshold value.

    Return
    ------
    img_th: np.ndarray (M, N)
        Image after applying threshold.
    """

    # Define default value for input image
    img_th = np.zeros_like(img)
    img_th = img / np.max(img) * 255
    # ------------------
    # Your code here ... 
    img_th = img_th > threshold
    # ------------------

    return img_th

def apply_gaussian(img, sigma):
    """
    Apply gaussian filter to input image.

    Args
    ----
    img: np.ndarray (M, N)
        Image to apply gaussian filter.
    sigma: float
        Standard deviation for gaussian filter.

    Return
    ------
    img_gaussian: np.ndarray (M, N)
        Image after applying gaussian filter.
    """

    # Define default value for input image
    img_gaussian = np.zeros_like(img)
    # ------------------
    # Your code here ... 
    img_gaussian = sk.filters.gaussian(img, sigma=sigma)
    # ------------------

    return img_gaussian

def getThresholdedImage(imgGray):
    low_pass = cv.GaussianBlur(imgGray, (31, 31), 0)
    high_pass = cv.subtract(imgGray, low_pass) 
    inverted_HP = (255 - high_pass)
    _, inverted_thresholded = cv.threshold(inverted_HP, 251, 255, cv.THRESH_BINARY)
    _, thresholded = cv.threshold(high_pass, 4, 255, cv.THRESH_BINARY)

    return inverted_thresholded, thresholded


In [ ]:
TEST = "NOISY_BG" # HAND, NOISY_BG, NEUTRAL_BG

if TEST == "NEUTRAL_BG":
    img = cv.imread('../project/data/train/1. neutral_bg/L1010311.JPG')

elif TEST == "HAND":
    img = cv.imread('../project/data/train/6. hand_outliers/L1010521.JPG')

else:
    img = cv.imread('../project/data/train/2. noisy_bg/L1010328.JPG')


img = cv.resize(img, (0,0), fx=0.25, fy=0.25)

imgRGB = cv.cvtColor(img, cv.COLOR_BGR2RGB)
imgGray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

display_img_side(imgRGB, imgGray)

In [363]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import os
import pandas as pd
def sas(idx, img, size = 45, sigma = 2.7, thres1 = 247, thres2 = 6):

    std = np.std(cv.cvtColor(255 -img, cv.COLOR_BGR2GRAY))
    print('variance:', std)
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    img = cv.resize(img, (0,0), fx=0.25, fy=0.25)
    img_copy = img.copy()

    img[:,:,0] = img[:,:,0]*0.2 #red
    img[:,:,1] = img[:,:,1]*0.2 #green
    img[:,:,2] = img[:,:,2]*1 #blue 

    red_mean = np.mean(img_copy[:,:,0]) 

    imgGray = cv.cvtColor(255 - img, cv.COLOR_BGR2GRAY)

    if std < 18: #neutral
        remove_objects_size = 10
        thres2 = 1
        sigma = 2.7
        p2 = 32
        th_sigma = 2
        open_th = 3
    
    else: #hand
        remove_objects_size = 120
        if red_mean > 184:
            thres2 = 4
            sigma = 2.7
            p2 = 32
            th_sigma = 2.5
            open_th = 6
        else: # noisy
            thres2 = 1
            sigma = 2.7
            p2 = 31
            th_sigma = 1.8
            open_th = 4

    low_pass = cv.GaussianBlur(imgGray, (size, size), sigma)
    high_pass = cv.subtract(imgGray, low_pass) 
    inverted_HP = (255 - high_pass)
    _, inverted_thresholded = cv.threshold(inverted_HP, thres1, 255, cv.THRESH_BINARY)
    _, thresholded = cv.threshold(high_pass, thres2, 255, cv.THRESH_BINARY)

    #thresholded = np.uint8(remove_objects(thresholded, 16) * 255)
    thresholded = np.uint8(remove_objects(thresholded,remove_objects_size) * 255)
    thresholded_open = apply_closing(thresholded, open_th) # 3
    thresholded_open = cv.GaussianBlur(thresholded_open, (size, size), th_sigma) # 2

    circles = cv.HoughCircles(thresholded_open, cv.HOUGH_GRADIENT, dp=1, minDist=50, param1=15, param2=p2, minRadius=40, maxRadius=120) # 32
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for i in circles[0, :]:
            cv.circle(img_copy, (i[0], i[1]), i[2], (0, 0, 0), 10)

    display_img_side(thresholded, img_copy,idx)

# set the default values for the parameters in the interactive plot
#interact(sas, size=(25, 51, 2), sigma=(0, 2, 0.1), thres1=(200, 255, 1), thres2=(1, 20, 1)) 

# apply the function to all the images in the folder
def process_images(images):
    means = []
    stds = []
    mean_R_list = []
    mean_G_list = []
    mean_B_list = []
    for idx, image in enumerate(images):
        img_RGB = cv.cvtColor(image, cv.COLOR_BGR2RGB)
        sas(idx,image)
    #     mean_R_list.append(np.mean(img_RGB[:,:,0]))
    #     mean_G_list.append(np.mean(img_RGB[:,:,1]))
    #     mean_B_list.append(np.mean(img_RGB[:,:,2]))
    #     imgGray = cv.cvtColor(255 - image, cv.COLOR_BGR2GRAY) 
    #     means.append(np.mean(imgGray))
    #     stds.append(np.std(imgGray))
    # df_means_stds = pd.DataFrame({'mean': means, 'std': stds, 'mean_R': mean_R_list, 'mean_G': mean_G_list, 'mean_B': mean_B_list})
    # return df_means_stds

# 45 1.7 247 12

In [222]:
hand_images_list = os.listdir('../project/data/train/3. hand')
hand_images_path = '../project/data/train/3. hand'
hand_outliers_images_list = os.listdir('../project/data/train/6. hand_outliers')
hand_outliers_images_path = '../project/data/train/6. hand_outliers'
neutral_images_list = os.listdir('../project/data/train/1. neutral_bg')
neutral_images_path = '../project/data/train/1. neutral_bg'
neutral_images_outliers_list = os.listdir('../project/data/train/4. neutral_bg_outliers')
neutral_images_outliers_path = '../project/data/train/4. neutral_bg_outliers'
noisy_images_list = os.listdir('../project/data/train/2. noisy_bg')
noisy_images_path = '../project/data/train/2. noisy_bg'
noisy_images_outliers_list = os.listdir('../project/data/train/5. noisy_bg_outliers')
noisy_images_outliers_path = '../project/data/train/5. noisy_bg_outliers'

def load_images(image_list, path):
    images = []
    for image in image_list:
        img = cv.imread(os.path.join(path, image))
        images.append(img)
    return images

hand_images = load_images(hand_images_list, hand_images_path)
hand_outliers_images = load_images(hand_outliers_images_list, hand_outliers_images_path)
neutral_images = load_images(neutral_images_list, neutral_images_path)
neutral_images_outliers = load_images(neutral_images_outliers_list, neutral_images_outliers_path)
noisy_images = load_images(noisy_images_list, noisy_images_path)
noisy_images_outliers = load_images(noisy_images_outliers_list, noisy_images_outliers_path)


In [369]:
test_images_list = os.listdir('../project/data/test')
test_images_list = [image for image in test_images_list if '.JPG' in image]
test_images_path = '../project/data/test'
test_images = load_images(test_images_list, test_images_path)

In [370]:
df_test_images = process_images(test_images)

variance: 28.916838852914207
variance: 28.14976183954491
variance: 19.544983533958437
variance: 23.0137559695069
variance: 26.883820688835982
variance: 22.185362794790056
variance: 14.100518294153536
variance: 26.733503152868167
variance: 27.812481981003096
variance: 8.84090720798851
variance: 15.740451293215505
variance: 28.473955526781477
variance: 14.287310464589785
variance: 27.58608799961319
variance: 15.185211565056287
variance: 28.163397314956548
variance: 24.39353672215666
variance: 14.691178633771417
variance: 23.692191836282465
variance: 14.972772021119606
variance: 12.051588543422392
variance: 15.126080046785367
variance: 21.963151345593868
variance: 26.166538742651277
variance: 21.459701677571985
variance: 27.307053187653
variance: 27.692785358182764
variance: 10.766144104766488
variance: 13.8698467341139
variance: 14.211239979074653
variance: 27.342725550284392
variance: 12.170778855196742
variance: 28.24329937479452
variance: 20.994462508228906
variance: 22.38748202235018

In [365]:
print('HAND')
#df_hand_outliers = process_images(hand_outliers_images)
#df_hand = process_images(hand_images)
# print('NEUTRAL')
#df_neutral = process_images(neutral_images)
# df_neutral_outliers = process_images(neutral_images_outliers)
# print('NOISY')
#df_noisy = process_images(noisy_images)
# df_noisy_outliers = process_images(noisy_images_outliers)

HAND
variance: 31.699752789032114
variance: 27.437237869741598
variance: 27.46636207873105
variance: 29.129590990581736
variance: 27.949042551460185
variance: 28.078185656822974
variance: 27.898555326754863
variance: 26.824059358957392
variance: 28.647441924726575
variance: 26.93692535558631
variance: 26.783117181526332
variance: 28.61145906912125
variance: 27.671223805688697
variance: 28.415815335772358
variance: 27.872526163002558


In [352]:
display(df_neutral.describe())
display(df_neutral_outliers.describe())
display(df_hand_outliers.describe())
display(df_hand.describe())
display(df_noisy.describe())
display(df_noisy_outliers.describe())

,mean,std,mean_R,mean_G,mean_B
count,16.000000,16.000000,16.000000,16.000000,16.000000
mean,57.993373,14.050517,202.656448,195.915048,187.948148
std,2.904547,1.548617,2.715235,2.965229,3.304457
min,56.042754,11.282972,192.760312,185.282787,176.653117
25%,56.909366,13.380052,202.728515,195.983652,187.836787
50%,57.355690,13.751663,203.226228,196.603455,188.601007
75%,57.798131,14.645607,203.617696,197.019417,189.506441
max,68.484102,17.209183,205.221483,197.963777,191.205809


,mean,std,mean_R,mean_G,mean_B
count,17.000000,17.000000,17.000000,17.000000,17.000000
mean,73.605633,11.518221,187.572538,180.172483,171.893539
std,0.614470,2.278881,0.598802,0.709542,1.206369
min,72.286267,7.906520,186.437016,179.366821,169.652767
25%,73.443010,9.918522,187.295156,179.633484,171.224084
50%,73.783624,11.916464,187.364840,180.127704,171.989694
75%,74.085508,12.397106,187.729834,180.460196,172.254213
max,74.274572,15.084879,188.734620,181.651046,174.184288


,mean,std,mean_R,mean_G,mean_B
count,7.000000,7.000000,7.000000,7.000000,7.000000
mean,75.808792,21.273094,189.546766,175.564825,170.505872
std,3.306609,1.359783,3.140607,3.395315,3.543157
min,73.289551,19.395056,182.995616,168.154465,162.958423
25%,74.061035,20.404995,189.195623,175.900913,170.475148
50%,75.206217,21.262331,190.290244,176.238195,170.939777
75%,75.528705,22.032672,191.286284,177.267018,172.597152
max,82.986298,23.378934,192.577687,178.225254,173.498302


,mean,std,mean_R,mean_G,mean_B
count,10.000000,10.000000,10.000000,10.000000,10.000000
mean,70.417288,20.793032,197.509959,179.942362,174.574490
std,2.753980,2.146482,3.506088,2.788293,3.325977
min,66.935255,18.029392,189.293718,172.834135,169.698133
25%,69.328125,18.800177,196.131088,179.806394,171.748003
50%,70.043870,20.903021,198.512829,180.429435,174.602992
75%,70.288051,22.343532,200.013321,180.794951,176.860549
max,77.574299,24.460310,200.513134,183.870010,179.985442


,mean,std,mean_R,mean_G,mean_B
count,15.000000,15.000000,15.000000,15.000000,15.000000
mean,73.641712,28.094753,177.197346,184.862874,174.248268
std,1.067166,1.211080,2.944717,0.892737,2.831441
min,71.462563,26.783117,171.253976,183.096255,166.570521
25%,72.970748,27.451800,175.650887,184.425054,173.787600
50%,73.611302,27.898555,176.724090,185.067460,175.344022
75%,74.145542,28.513637,178.924691,185.303043,175.733477
max,76.193777,31.699753,182.901190,186.533816,176.538107


,mean,std,mean_R,mean_G,mean_B
count,16.000000,16.000000,16.000000,16.000000,16.000000
mean,79.491679,27.207227,171.899431,179.133701,166.300146
std,0.574543,0.431399,1.384970,0.547036,1.589707
min,78.285271,26.179893,168.983463,178.507374,162.056605
25%,79.171737,26.970506,171.015344,178.686911,165.389853
50%,79.515100,27.404874,171.674594,178.970635,166.421784
75%,79.880111,27.502892,172.580684,179.476814,167.644066
max,80.322466,27.809644,174.878942,180.063153,168.414760
